In [53]:
import re
import json
import numpy as np
import pandas as pd
import jieba
from zhconv import convert
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

In [54]:
# 读入20220821所有数据
all_data = pd.read_csv('../data/all_sample_20220821_spark.csv').drop(['Unnamed: 0'], axis=1)

/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_38964/3330071591.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data = pd.read_csv('../data/all_sample_20220821_spark.csv').drop(['Unnamed: 0'], axis=1)


In [55]:
# 显示所有的列
all_data.columns

Index(['cv_id', 'jd_id', 'jd_code', 'title', 'deadline', 'category_id',
       'category_name', 'company_id', 'company_name', 'recruit_number',
       'work_age', 'min_annual_salary', 'max_annual_salary',
       'min_month_salary', 'max_month_salary', 'pay_months', 'created_by_name',
       'created_by', 'created_at', 'updated_by', 'updated_at', 'tenant_id',
       'guarantee_period', 'status', 'recommended_rcn', 'tags', 'workplaces',
       'description', 'school', 'degree', 'post_level_id', 'requirement', 'ds',
       '_id', 'additionInfo', 'age', 'certificates', 'code', 'competenceScore',
       'completeness', 'countryCode', 'createdAt', 'createdBy',
       'currentCompany', 'currentPosition', 'currentSalary', 'dateOfBirth',
       'degree.1', 'desiredIndustry', 'desiredPosition', 'desiredSalary',
       'dutyTime', 'expectLocation', 'gender', 'importType', 'industry',
       'interviewTime', 'jobIntentStatus', 'languageSkills', 'languages',
       'location', 'maintainerId', 'majo

In [56]:
# 注意一下，这里的列名和sample pipeline中的列名有一点大小写区别，问题不大
cv_columns = ['cv_id', 'currentPosition', 'desiredPosition', 'industry', 'desiredIndustry', 'majorName', 'skills', 'eduTracks', 'jobTracks', 'projectTracks']
jd_columns = ['jd_id', 'title', 'category_name', 'tags', 'description', 'requirement']

In [57]:
jd_data = all_data[jd_columns]
jd_data

# 可以做3个向量
# 1.title + category_name + tags
# 2.description
# 3.requirement
# 每个向量的维度在20维左右

,jd_id,title,category_name,tags,description,requirement
0,2c9207157bcbb76b017bcde2b46d04cd,市场营销负责人,市场营销,"[""营销"",""管理"",""销售"",""战略规划"",""管理工作""]",1、岗位职责:2、主导制定公司营销战略规划并参与公司整体战略规划制定。3、全面管理公司的市场...,1、电商2、线下营销
1,2c9207157bcbb76b017bf7f2bbbb3153,会员管理经理,用户运营,"[""会员运营"",""会员管理"",""管理"",""营销"",""社群运营""]",1、岗位职责：2、1. 负责“九木杂物社”品牌全渠道会员体系的管理，规划品牌会员运营方向，包...,1、有会员管理、会员活动经验
2,2c9207157bf94d99017c108f769405e0,产品市场经理,产品经理,"[""EXCEL"",""OFFICE"",""PPT"",""UI""]",1、•Support regional strategic planning and mar...,1、find jd
3,2c9207157d2f0ec5017d5a9265ba33d9,海外区域运营经理/高级经理,运营,"[""管理"",""运营"",""团队管理"",""电商"",""游戏""]",1、负责短视频业务的运营策略制定，配合完成公司的营收业务指标；2、主导团队的业务方向，包括但...,1、本科及以上学历优先，3年以上社交、直播、游戏、电商或相关行业运营工作经验；2、英语听说读...
4,2c9207157d631abb017d6eed1c79139f,高级项目经理,高级管理,"[""项目管理"",""推广"",""归档"",""风险评估"",""交货""]",1、负责从产品建立到第一批生产交付的全过程项目管理;2、 负责实现产品要求、进度、成本、质量...,1、 本科及以上学历，计算机科学或相关专业。能用英语交流者优先;2、5年以上可穿戴相关产品项...
...,...,...,...,...,...,...
81291,8a69f6f8823a93b901823d891b761d51,技术经理,机械设备工程师,"[""材料"",""考核"",""维修"",""技术标准"",""不锈钢""]",职责描述：1、全面负责公司技术标准制、各岗位的工艺制定跟改善；2、重要设备参数收集管控包各棍...,任职要求：1、具备压延400系不锈钢材料技术相关工作经验，能对水扎四六棍扎异常原因分析解决、...
81292,8a69c468826ee9b40182831bedfc5e79,中央市场总监,市场总监,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...
81293,8a69f6f8823a93b9018247c30aca4e3d,中央市场总监,市场/营销,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...
81294,2c92071580dd3ab70180f18714e119c0,java开发工程师（不限方向）,Java,"[""java"",""微服务/性能优化"",""SQL"",""架构"",""需求分析""]",1、负责公司OA产品的功能规划、需求分析设计、技术实现和用户体验；2、负责所属模块的代码开发...,1、本科及以上学历，2年以上的Java开发及应用经验，计算机或者相关专业;2、良好的问题理解...


##### 将title + category_name + tags合并成一个字段

In [58]:
# step1: 缺失值填充

jd_data['title'].fillna('', inplace=True)
jd_data['category_name'].fillna('', inplace=True)
jd_data['tags'].fillna('[]', inplace=True)

/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_38964/2002744044.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jd_data['title'].fillna('', inplace=True)
/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_38964/2002744044.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jd_data['category_name'].fillna('', inplace=True)
/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_38964/2002744044.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

In [59]:
# step2: 定义新列，将title + category_name合并

col_name1 = 'title'
col_name2 = 'category_name'

def col_merge_str_fun(series):
    '''
    适用于文本列合并
    '''
    return series[col_name1] + ' ' + series[col_name2]

jd_data['title_category'] = jd_data.apply(col_merge_str_fun, axis=1)
jd_data

/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_38964/2104666904.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jd_data['title_category'] = jd_data.apply(col_merge_str_fun, axis=1)


,jd_id,title,category_name,tags,description,requirement,title_category
0,2c9207157bcbb76b017bcde2b46d04cd,市场营销负责人,市场营销,"[""营销"",""管理"",""销售"",""战略规划"",""管理工作""]",1、岗位职责:2、主导制定公司营销战略规划并参与公司整体战略规划制定。3、全面管理公司的市场...,1、电商2、线下营销,市场营销负责人 市场营销
1,2c9207157bcbb76b017bf7f2bbbb3153,会员管理经理,用户运营,"[""会员运营"",""会员管理"",""管理"",""营销"",""社群运营""]",1、岗位职责：2、1. 负责“九木杂物社”品牌全渠道会员体系的管理，规划品牌会员运营方向，包...,1、有会员管理、会员活动经验,会员管理经理 用户运营
2,2c9207157bf94d99017c108f769405e0,产品市场经理,产品经理,"[""EXCEL"",""OFFICE"",""PPT"",""UI""]",1、•Support regional strategic planning and mar...,1、find jd,产品市场经理 产品经理
3,2c9207157d2f0ec5017d5a9265ba33d9,海外区域运营经理/高级经理,运营,"[""管理"",""运营"",""团队管理"",""电商"",""游戏""]",1、负责短视频业务的运营策略制定，配合完成公司的营收业务指标；2、主导团队的业务方向，包括但...,1、本科及以上学历优先，3年以上社交、直播、游戏、电商或相关行业运营工作经验；2、英语听说读...,海外区域运营经理/高级经理 运营
4,2c9207157d631abb017d6eed1c79139f,高级项目经理,高级管理,"[""项目管理"",""推广"",""归档"",""风险评估"",""交货""]",1、负责从产品建立到第一批生产交付的全过程项目管理;2、 负责实现产品要求、进度、成本、质量...,1、 本科及以上学历，计算机科学或相关专业。能用英语交流者优先;2、5年以上可穿戴相关产品项...,高级项目经理 高级管理
...,...,...,...,...,...,...,...
81291,8a69f6f8823a93b901823d891b761d51,技术经理,机械设备工程师,"[""材料"",""考核"",""维修"",""技术标准"",""不锈钢""]",职责描述：1、全面负责公司技术标准制、各岗位的工艺制定跟改善；2、重要设备参数收集管控包各棍...,任职要求：1、具备压延400系不锈钢材料技术相关工作经验，能对水扎四六棍扎异常原因分析解决、...,技术经理 机械设备工程师
81292,8a69c468826ee9b40182831bedfc5e79,中央市场总监,市场总监,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...,中央市场总监 市场总监
81293,8a69f6f8823a93b9018247c30aca4e3d,中央市场总监,市场/营销,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...,中央市场总监 市场/营销
81294,2c92071580dd3ab70180f18714e119c0,java开发工程师（不限方向）,Java,"[""java"",""微服务/性能优化"",""SQL"",""架构"",""需求分析""]",1、负责公司OA产品的功能规划、需求分析设计、技术实现和用户体验；2、负责所属模块的代码开发...,1、本科及以上学历，2年以上的Java开发及应用经验，计算机或者相关专业;2、良好的问题理解...,java开发工程师（不限方向） Java


In [60]:
# step3 将新列title_category通过jieba分词处理成列表

col_name = 'title_category'

def col_jieba_fun(series):
    col = series[col_name]

    # 字符串变列表
    if col.startswith("[") and col.endswith("]"):
        col = json.loads(col)
    else:
        col = re.split(",|，|/| ", col)

    # 对于中文，进入jieba前不需要添加空格
    # 不过，如果是中英文混合，就必须空格了
    col_str = " ".join(col)
    col_list = jieba.lcut(col_str, cut_all=False)
    return col_list

jd_data['title_category_jieba'] = jd_data.apply(col_jieba_fun, axis=1)
jd_data


/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_38964/2374843942.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jd_data['title_category_jieba'] = jd_data.apply(col_jieba_fun, axis=1)


,jd_id,title,category_name,tags,description,requirement,title_category,title_category_jieba
0,2c9207157bcbb76b017bcde2b46d04cd,市场营销负责人,市场营销,"[""营销"",""管理"",""销售"",""战略规划"",""管理工作""]",1、岗位职责:2、主导制定公司营销战略规划并参与公司整体战略规划制定。3、全面管理公司的市场...,1、电商2、线下营销,市场营销负责人 市场营销,"[市场营销, 负责人, , 市场营销]"
1,2c9207157bcbb76b017bf7f2bbbb3153,会员管理经理,用户运营,"[""会员运营"",""会员管理"",""管理"",""营销"",""社群运营""]",1、岗位职责：2、1. 负责“九木杂物社”品牌全渠道会员体系的管理，规划品牌会员运营方向，包...,1、有会员管理、会员活动经验,会员管理经理 用户运营,"[会员, 管理, 经理, , 用户, 运营]"
2,2c9207157bf94d99017c108f769405e0,产品市场经理,产品经理,"[""EXCEL"",""OFFICE"",""PPT"",""UI""]",1、•Support regional strategic planning and mar...,1、find jd,产品市场经理 产品经理,"[产品, 市场, 经理, , 产品, 经理]"
3,2c9207157d2f0ec5017d5a9265ba33d9,海外区域运营经理/高级经理,运营,"[""管理"",""运营"",""团队管理"",""电商"",""游戏""]",1、负责短视频业务的运营策略制定，配合完成公司的营收业务指标；2、主导团队的业务方向，包括但...,1、本科及以上学历优先，3年以上社交、直播、游戏、电商或相关行业运营工作经验；2、英语听说读...,海外区域运营经理/高级经理 运营,"[海外, 区域, 运营, 经理, , 高级, 经理, , 运营]"
4,2c9207157d631abb017d6eed1c79139f,高级项目经理,高级管理,"[""项目管理"",""推广"",""归档"",""风险评估"",""交货""]",1、负责从产品建立到第一批生产交付的全过程项目管理;2、 负责实现产品要求、进度、成本、质量...,1、 本科及以上学历，计算机科学或相关专业。能用英语交流者优先;2、5年以上可穿戴相关产品项...,高级项目经理 高级管理,"[高级, 项目经理, , 高级, 管理]"
...,...,...,...,...,...,...,...,...
81291,8a69f6f8823a93b901823d891b761d51,技术经理,机械设备工程师,"[""材料"",""考核"",""维修"",""技术标准"",""不锈钢""]",职责描述：1、全面负责公司技术标准制、各岗位的工艺制定跟改善；2、重要设备参数收集管控包各棍...,任职要求：1、具备压延400系不锈钢材料技术相关工作经验，能对水扎四六棍扎异常原因分析解决、...,技术经理 机械设备工程师,"[技术, 经理, , 机械设备, 工程师]"
81292,8a69c468826ee9b40182831bedfc5e79,中央市场总监,市场总监,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...,中央市场总监 市场总监,"[中央, 市场总监, , 市场总监]"
81293,8a69f6f8823a93b9018247c30aca4e3d,中央市场总监,市场/营销,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...,中央市场总监 市场/营销,"[中央, 市场总监, , 市场, , 营销]"
81294,2c92071580dd3ab70180f18714e119c0,java开发工程师（不限方向）,Java,"[""java"",""微服务/性能优化"",""SQL"",""架构"",""需求分析""]",1、负责公司OA产品的功能规划、需求分析设计、技术实现和用户体验；2、负责所属模块的代码开发...,1、本科及以上学历，2年以上的Java开发及应用经验，计算机或者相关专业;2、良好的问题理解...,java开发工程师（不限方向） Java,"[java, 开发, 工程师, （, 不, 限, 方向, ）, , Java]"


In [61]:
# step4 将tags列通过jieba分词处理成列表

col_name = 'tags'

def col_jieba_fun(series):
    col = series[col_name]

    # 字符串变列表
    if col.startswith("[") and col.endswith("]"):
        col = json.loads(col)
    else:
        col = re.split(",|，|/| ", col)

    # 对于中文，进入jieba前不需要添加空格
    # 不过，如果是中英文混合，就必须空格了
    col_str = " ".join(col)
    col_list = jieba.lcut(col_str, cut_all=False)
    return col_list

jd_data['tags_jieba'] = jd_data.apply(col_jieba_fun, axis=1)
jd_data


/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_38964/1798798320.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jd_data['tags_jieba'] = jd_data.apply(col_jieba_fun, axis=1)


,jd_id,title,category_name,tags,description,requirement,title_category,title_category_jieba,tags_jieba
0,2c9207157bcbb76b017bcde2b46d04cd,市场营销负责人,市场营销,"[""营销"",""管理"",""销售"",""战略规划"",""管理工作""]",1、岗位职责:2、主导制定公司营销战略规划并参与公司整体战略规划制定。3、全面管理公司的市场...,1、电商2、线下营销,市场营销负责人 市场营销,"[市场营销, 负责人, , 市场营销]","[营销, , 管理, , 销售, , 战略规划, , 管理工作]"
1,2c9207157bcbb76b017bf7f2bbbb3153,会员管理经理,用户运营,"[""会员运营"",""会员管理"",""管理"",""营销"",""社群运营""]",1、岗位职责：2、1. 负责“九木杂物社”品牌全渠道会员体系的管理，规划品牌会员运营方向，包...,1、有会员管理、会员活动经验,会员管理经理 用户运营,"[会员, 管理, 经理, , 用户, 运营]","[会员, 运营, , 会员, 管理, , 管理, , 营销, , 社群, 运营]"
2,2c9207157bf94d99017c108f769405e0,产品市场经理,产品经理,"[""EXCEL"",""OFFICE"",""PPT"",""UI""]",1、•Support regional strategic planning and mar...,1、find jd,产品市场经理 产品经理,"[产品, 市场, 经理, , 产品, 经理]","[EXCEL, , OFFICE, , PPT, , UI]"
3,2c9207157d2f0ec5017d5a9265ba33d9,海外区域运营经理/高级经理,运营,"[""管理"",""运营"",""团队管理"",""电商"",""游戏""]",1、负责短视频业务的运营策略制定，配合完成公司的营收业务指标；2、主导团队的业务方向，包括但...,1、本科及以上学历优先，3年以上社交、直播、游戏、电商或相关行业运营工作经验；2、英语听说读...,海外区域运营经理/高级经理 运营,"[海外, 区域, 运营, 经理, , 高级, 经理, , 运营]","[管理, , 运营, , 团队, 管理, , 电商, , 游戏]"
4,2c9207157d631abb017d6eed1c79139f,高级项目经理,高级管理,"[""项目管理"",""推广"",""归档"",""风险评估"",""交货""]",1、负责从产品建立到第一批生产交付的全过程项目管理;2、 负责实现产品要求、进度、成本、质量...,1、 本科及以上学历，计算机科学或相关专业。能用英语交流者优先;2、5年以上可穿戴相关产品项...,高级项目经理 高级管理,"[高级, 项目经理, , 高级, 管理]","[项目管理, , 推广, , 归档, , 风险, 评估, , 交货]"
...,...,...,...,...,...,...,...,...,...
81291,8a69f6f8823a93b901823d891b761d51,技术经理,机械设备工程师,"[""材料"",""考核"",""维修"",""技术标准"",""不锈钢""]",职责描述：1、全面负责公司技术标准制、各岗位的工艺制定跟改善；2、重要设备参数收集管控包各棍...,任职要求：1、具备压延400系不锈钢材料技术相关工作经验，能对水扎四六棍扎异常原因分析解决、...,技术经理 机械设备工程师,"[技术, 经理, , 机械设备, 工程师]","[材料, , 考核, , 维修, , 技术标准, , 不锈钢]"
81292,8a69c468826ee9b40182831bedfc5e79,中央市场总监,市场总监,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...,中央市场总监 市场总监,"[中央, 市场总监, , 市场总监]","[市场推广, , 医疗器械, , 语言, , 材料, , 营销]"
81293,8a69f6f8823a93b9018247c30aca4e3d,中央市场总监,市场/营销,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...,中央市场总监 市场/营销,"[中央, 市场总监, , 市场, , 营销]","[市场推广, , 医疗器械, , 语言, , 材料, , 营销]"
81294,2c92071580dd3ab70180f18714e119c0,java开发工程师（不限方向）,Java,"[""java"",""微服务/性能优化"",""SQL"",""架构"",""需求分析""]",1、负责公司OA产品的功能规划、需求分析设计、技术实现和用户体验；2、负责所属模块的代码开发...,1、本科及以上学历，2年以上的Java开发及应用经验，计算机或者相关专业;2、良好的问题理解...,java开发工程师（不限方向） Java,"[java, 开发, 工程师, （, 不, 限, 方向, ）, , Java]","[java, , 微, 服务, /, 性能, 优化, , SQL, , 架构, , ..."


In [63]:
# step5 将title_category_jieba和tags_jieba两个列表合并

col_name1 = 'title_category_jieba'
col_name2 = 'tags_jieba'

def col_merge_list_fun(series):
    '''
    适用于列表列合并
    '''
    return series[col_name1] + series[col_name2]

jd_data['title_category_tags_jieba'] = jd_data.apply(col_merge_list_fun, axis=1)
jd_data


/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_38964/1665803803.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jd_data['title_category_tags_jieba'] = jd_data.apply(col_merge_list_fun, axis=1)


,jd_id,title,category_name,tags,description,requirement,title_category,title_category_jieba,tags_jieba,title_category_tags_jieba
0,2c9207157bcbb76b017bcde2b46d04cd,市场营销负责人,市场营销,"[""营销"",""管理"",""销售"",""战略规划"",""管理工作""]",1、岗位职责:2、主导制定公司营销战略规划并参与公司整体战略规划制定。3、全面管理公司的市场...,1、电商2、线下营销,市场营销负责人 市场营销,"[市场营销, 负责人, , 市场营销]","[营销, , 管理, , 销售, , 战略规划, , 管理工作]","[市场营销, 负责人, , 市场营销, 营销, , 管理, , 销售, , 战略规划..."
1,2c9207157bcbb76b017bf7f2bbbb3153,会员管理经理,用户运营,"[""会员运营"",""会员管理"",""管理"",""营销"",""社群运营""]",1、岗位职责：2、1. 负责“九木杂物社”品牌全渠道会员体系的管理，规划品牌会员运营方向，包...,1、有会员管理、会员活动经验,会员管理经理 用户运营,"[会员, 管理, 经理, , 用户, 运营]","[会员, 运营, , 会员, 管理, , 管理, , 营销, , 社群, 运营]","[会员, 管理, 经理, , 用户, 运营, 会员, 运营, , 会员, 管理, , ..."
2,2c9207157bf94d99017c108f769405e0,产品市场经理,产品经理,"[""EXCEL"",""OFFICE"",""PPT"",""UI""]",1、•Support regional strategic planning and mar...,1、find jd,产品市场经理 产品经理,"[产品, 市场, 经理, , 产品, 经理]","[EXCEL, , OFFICE, , PPT, , UI]","[产品, 市场, 经理, , 产品, 经理, EXCEL, , OFFICE, , P..."
3,2c9207157d2f0ec5017d5a9265ba33d9,海外区域运营经理/高级经理,运营,"[""管理"",""运营"",""团队管理"",""电商"",""游戏""]",1、负责短视频业务的运营策略制定，配合完成公司的营收业务指标；2、主导团队的业务方向，包括但...,1、本科及以上学历优先，3年以上社交、直播、游戏、电商或相关行业运营工作经验；2、英语听说读...,海外区域运营经理/高级经理 运营,"[海外, 区域, 运营, 经理, , 高级, 经理, , 运营]","[管理, , 运营, , 团队, 管理, , 电商, , 游戏]","[海外, 区域, 运营, 经理, , 高级, 经理, , 运营, 管理, , 运营, ..."
4,2c9207157d631abb017d6eed1c79139f,高级项目经理,高级管理,"[""项目管理"",""推广"",""归档"",""风险评估"",""交货""]",1、负责从产品建立到第一批生产交付的全过程项目管理;2、 负责实现产品要求、进度、成本、质量...,1、 本科及以上学历，计算机科学或相关专业。能用英语交流者优先;2、5年以上可穿戴相关产品项...,高级项目经理 高级管理,"[高级, 项目经理, , 高级, 管理]","[项目管理, , 推广, , 归档, , 风险, 评估, , 交货]","[高级, 项目经理, , 高级, 管理, 项目管理, , 推广, , 归档, , 风..."
...,...,...,...,...,...,...,...,...,...,...
81291,8a69f6f8823a93b901823d891b761d51,技术经理,机械设备工程师,"[""材料"",""考核"",""维修"",""技术标准"",""不锈钢""]",职责描述：1、全面负责公司技术标准制、各岗位的工艺制定跟改善；2、重要设备参数收集管控包各棍...,任职要求：1、具备压延400系不锈钢材料技术相关工作经验，能对水扎四六棍扎异常原因分析解决、...,技术经理 机械设备工程师,"[技术, 经理, , 机械设备, 工程师]","[材料, , 考核, , 维修, , 技术标准, , 不锈钢]","[技术, 经理, , 机械设备, 工程师, 材料, , 考核, , 维修, , 技术..."
81292,8a69c468826ee9b40182831bedfc5e79,中央市场总监,市场总监,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...,中央市场总监 市场总监,"[中央, 市场总监, , 市场总监]","[市场推广, , 医疗器械, , 语言, , 材料, , 营销]","[中央, 市场总监, , 市场总监, 市场推广, , 医疗器械, , 语言, , 材..."
81293,8a69f6f8823a93b9018247c30aca4e3d,中央市场总监,市场/营销,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...,中央市场总监 市场/营销,"[中央, 市场总监, , 市场, , 营销]","[市场推广, , 医疗器械, , 语言, , 材料, , 营销]","[中央, 市场总监, , 市场, , 营销, 市场推广, , 医疗器械, , 语言,..."
81294,2c92071580dd3ab70180f18714e119c0,java开发工程师（不限方向）,Java,"[""java"",""微服务/性能优化"",""SQL"",""架构"",""需求分析""]",1、负责公司OA产品的功能规划、需求分析设计、技术实现和用户体验；2、负责所属模块的代码开发...,1、本科及以上学历，2年以上的Java开发及应用经验，计算机或者相关专业;2、良好的问题理解...,java开发工程师（不限方向） Java,"[java, 开发, 工程师, （, 不, 限, 方向, ）, , Java]","[java, , 微, 服务, /, 性能, 优化, , SQL, , 架构, , ...","[java, 开发, 工程师, （, 不, 限, 方向, ）, , Java, java,..."


In [64]:
#step6 过滤title_category_tags_jieba, 得到以空格分割的字符串

col_name = 'title_category_tags_jieba'
def col_jieba_filter_fun(series):

    col_list_filter = []
    # 得到tag列表
    col_list = series[col_name]
    # print(tags_list[0])

    pun_masks_english = [",", ".", "/", "[", "]", "{", "}", "(", ")", ":", "*", "#", "!", " ", "\"", "\\"]
    pun_masks_chinese = ["，", "。", "、", "（", "）", "：", "！", "”", "“"]
    pun_masks = pun_masks_english + pun_masks_chinese

    # 过滤
    for tag in col_list:
        # 转中文简体
        tag = convert(tag, "zh-hans")
        # 转英文小写
        tag = tag.lower()

        # 过滤数字
        if tag.isdigit():
            continue
        
        # 过滤单个字符
        if len(tag) <= 1:
            continue
        
        # 过滤标点
        flag = 1
        for pun in pun_masks:
            if pun in tag:
                flag = 0
                break
        if flag == 1:
            col_list_filter.append(tag)
    return " ".join(col_list_filter)

jd_data['title_category_tags_jieba_filter'] = jd_data.apply(col_jieba_filter_fun, axis=1)
jd_data

/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_38964/1629471737.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jd_data['title_category_tags_jieba_filter'] = jd_data.apply(col_jieba_filter_fun, axis=1)


,jd_id,title,category_name,tags,description,requirement,title_category,title_category_jieba,tags_jieba,title_category_tags_jieba,title_category_tags_jieba_filter
0,2c9207157bcbb76b017bcde2b46d04cd,市场营销负责人,市场营销,"[""营销"",""管理"",""销售"",""战略规划"",""管理工作""]",1、岗位职责:2、主导制定公司营销战略规划并参与公司整体战略规划制定。3、全面管理公司的市场...,1、电商2、线下营销,市场营销负责人 市场营销,"[市场营销, 负责人, , 市场营销]","[营销, , 管理, , 销售, , 战略规划, , 管理工作]","[市场营销, 负责人, , 市场营销, 营销, , 管理, , 销售, , 战略规划...",市场营销 负责人 市场营销 营销 管理 销售 战略规划 管理工作
1,2c9207157bcbb76b017bf7f2bbbb3153,会员管理经理,用户运营,"[""会员运营"",""会员管理"",""管理"",""营销"",""社群运营""]",1、岗位职责：2、1. 负责“九木杂物社”品牌全渠道会员体系的管理，规划品牌会员运营方向，包...,1、有会员管理、会员活动经验,会员管理经理 用户运营,"[会员, 管理, 经理, , 用户, 运营]","[会员, 运营, , 会员, 管理, , 管理, , 营销, , 社群, 运营]","[会员, 管理, 经理, , 用户, 运营, 会员, 运营, , 会员, 管理, , ...",会员 管理 经理 用户 运营 会员 运营 会员 管理 管理 营销 社群 运营
2,2c9207157bf94d99017c108f769405e0,产品市场经理,产品经理,"[""EXCEL"",""OFFICE"",""PPT"",""UI""]",1、•Support regional strategic planning and mar...,1、find jd,产品市场经理 产品经理,"[产品, 市场, 经理, , 产品, 经理]","[EXCEL, , OFFICE, , PPT, , UI]","[产品, 市场, 经理, , 产品, 经理, EXCEL, , OFFICE, , P...",产品 市场 经理 产品 经理 excel office ppt ui
3,2c9207157d2f0ec5017d5a9265ba33d9,海外区域运营经理/高级经理,运营,"[""管理"",""运营"",""团队管理"",""电商"",""游戏""]",1、负责短视频业务的运营策略制定，配合完成公司的营收业务指标；2、主导团队的业务方向，包括但...,1、本科及以上学历优先，3年以上社交、直播、游戏、电商或相关行业运营工作经验；2、英语听说读...,海外区域运营经理/高级经理 运营,"[海外, 区域, 运营, 经理, , 高级, 经理, , 运营]","[管理, , 运营, , 团队, 管理, , 电商, , 游戏]","[海外, 区域, 运营, 经理, , 高级, 经理, , 运营, 管理, , 运营, ...",海外 区域 运营 经理 高级 经理 运营 管理 运营 团队 管理 电商 游戏
4,2c9207157d631abb017d6eed1c79139f,高级项目经理,高级管理,"[""项目管理"",""推广"",""归档"",""风险评估"",""交货""]",1、负责从产品建立到第一批生产交付的全过程项目管理;2、 负责实现产品要求、进度、成本、质量...,1、 本科及以上学历，计算机科学或相关专业。能用英语交流者优先;2、5年以上可穿戴相关产品项...,高级项目经理 高级管理,"[高级, 项目经理, , 高级, 管理]","[项目管理, , 推广, , 归档, , 风险, 评估, , 交货]","[高级, 项目经理, , 高级, 管理, 项目管理, , 推广, , 归档, , 风...",高级 项目经理 高级 管理 项目管理 推广 归档 风险 评估 交货
...,...,...,...,...,...,...,...,...,...,...,...
81291,8a69f6f8823a93b901823d891b761d51,技术经理,机械设备工程师,"[""材料"",""考核"",""维修"",""技术标准"",""不锈钢""]",职责描述：1、全面负责公司技术标准制、各岗位的工艺制定跟改善；2、重要设备参数收集管控包各棍...,任职要求：1、具备压延400系不锈钢材料技术相关工作经验，能对水扎四六棍扎异常原因分析解决、...,技术经理 机械设备工程师,"[技术, 经理, , 机械设备, 工程师]","[材料, , 考核, , 维修, , 技术标准, , 不锈钢]","[技术, 经理, , 机械设备, 工程师, 材料, , 考核, , 维修, , 技术...",技术 经理 机械设备 工程师 材料 考核 维修 技术标准 不锈钢
81292,8a69c468826ee9b40182831bedfc5e79,中央市场总监,市场总监,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...,中央市场总监 市场总监,"[中央, 市场总监, , 市场总监]","[市场推广, , 医疗器械, , 语言, , 材料, , 营销]","[中央, 市场总监, , 市场总监, 市场推广, , 医疗器械, , 语言, , 材...",中央 市场总监 市场总监 市场推广 医疗器械 语言 材料 营销
81293,8a69f6f8823a93b9018247c30aca4e3d,中央市场总监,市场/营销,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...,中央市场总监 市场/营销,"[中央, 市场总监, , 市场, , 营销]","[市场推广, , 医疗器械, , 语言, , 材料, , 营销]","[中央, 市场总监, , 市场, , 营销, 市场推广, , 医疗器械, , 语言,...",中央 市场总监 市场 营销 市场推广 医疗器械 语言 材料 营销
81294,2c92071580dd3ab70180f18714e119c0,java开发工程师（不限方向）,Java,"[""java"",""微服务/性能优化"",""SQL"",""架构"",""需求分析""]",1、负责公司OA产品的功能规划、需求分析设计、技术实现和用户体验；2、负责所属模块的代码开发...,1、本科及以上学历，2年以上的Java开发及应用经验，计算机或者相关专业;2、良好的问题理解...,java开发工程师（不限方向） Java,"[java, 开发, 工程师, （, 不, 限, 方向, ）, , Java]","[java, , 微, 服务, /, 性能, 优化, , SQL, , 架构, , ...","[java, 开发, 工程师, （, 不, 限, 方向, ）, , Java, java,...",java 开发 工程师 方向 java java 服务 性能 优化 sql 架构 需求 分析


In [65]:
#step7 将title_category_tags_jieba_filter转成tfidf向量

def get_tfidf(df, col_name):
    # col_name必须是以空格分割的字符串
    text = df[col_name]
    
    vectorizer = TfidfVectorizer()
    # 返回的是nparray
    vector = vectorizer.fit_transform(text)
    return pd.DataFrame(vector.toarray()), vectorizer

tfidf, vectorizer = get_tfidf(jd_data, 'title_category_tags_jieba_filter')
tfidf


,0,1,2,3,4,5,6,7,8,9,...,3597,3598,3599,3600,3601,3602,3603,3604,3605,3606
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81291,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81292,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81293,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [66]:
vectorizer.vocabulary_

{'市场营销': 1878,
 '负责人': 3210,
 '营销': 3076,
 '管理': 2826,
 '销售': 3454,
 '战略规划': 2035,
 '管理工作': 2832,
 '会员': 1136,
 '经理': 2904,
 '用户': 2633,
 '运营': 3336,
 '社群': 2753,
 '产品': 1053,
 '市场': 1872,
 'excel': 288,
 'office': 597,
 'ppt': 665,
 'ui': 856,
 '海外': 2505,
 '区域': 1421,
 '高级': 3594,
 '团队': 1598,
 '电商': 2647,
 '游戏': 2544,
 '项目经理': 3520,
 '项目管理': 3519,
 '推广': 2143,
 '归档': 1966,
 '风险': 3550,
 '评估': 3178,
 '交货': 1049,
 '阿里': 3473,
 '智能': 2321,
 '金融': 3422,
 '行业': 3101,
 '解决方案': 3134,
 '架构师': 2384,
 '公有': 1257,
 'it': 442,
 '技术支持': 2072,
 '数据库': 2227,
 'devops': 231,
 '架构': 2383,
 '售前': 1564,
 '软件产品': 3305,
 'os': 611,
 '沟通': 2459,
 '能力': 3023,
 '软件': 3304,
 '测试': 2493,
 '反馈': 1502,
 '通信': 3366,
 '专卖店': 948,
 '店长': 1931,
 '门店': 3464,
 '调配': 3206,
 '信息': 1187,
 '收集': 2187,
 '旅行': 2281,
 '攻略': 2191,
 '内容': 1279,
 '策划': 2809,
 '调度': 3200,
 'ip': 430,
 '声学': 1676,
 '工程师': 1855,
 '技术': 2067,
 '音频': 3515,
 'ws': 907,
 '审核': 1776,
 '耳机': 2996,
 '结构设计': 2916,
 '仿真': 1121,
 'dsp': 253,
 'cocos2d': 1

In [67]:
#step8 将tfidf降维到n维

def get_tfidf_pca(tfidf, n=20):
    pca = PCA(n_components=n)
    tfidf_pca = pca.fit_transform(tfidf)
    tfidf_pca = pd.DataFrame(tfidf_pca)
    return tfidf_pca

tfidf_pca = get_tfidf_pca(tfidf, 10)
tfidf_pca

,0,1,2,3,4,5,6,7,8,9
0,-0.042800,-0.158419,-0.004027,0.164218,-0.007663,0.003671,0.000458,-0.022639,-0.031584,-0.028167
1,-0.236844,0.085825,-0.230643,-0.025126,-0.002678,0.011263,0.020758,-0.037957,0.001932,-0.037604
2,-0.190874,0.194128,0.343113,0.055060,0.033952,0.041315,-0.054437,0.004835,-0.047984,0.074485
3,-0.372099,0.191217,-0.314976,-0.012887,-0.015023,0.001450,0.020659,0.001995,-0.028147,0.112229
4,0.005621,-0.057164,0.001188,-0.038638,0.003270,0.000521,-0.038803,-0.021172,-0.066060,-0.073352
...,...,...,...,...,...,...,...,...,...,...
81291,0.034486,-0.046806,0.050004,-0.045030,-0.024683,-0.012986,-0.034474,-0.014730,0.000553,-0.027135
81292,0.015164,-0.080300,0.009704,-0.010153,-0.032632,-0.000476,-0.043522,-0.024407,-0.043817,-0.032028
81293,-0.005332,-0.084559,0.012105,0.034578,-0.044205,0.004479,-0.038792,-0.023121,-0.047877,-0.014138
81294,0.547197,0.404912,-0.158608,0.200424,0.078477,0.038578,0.044430,-0.027809,0.014323,0.049222
